# Staff Planning Optimization Case Study
Case Study project submitted by Saurabh Pai & Abhinav Choudhary

### Business Objective:
An insurance company InsurePlus wants to find the optimal number of staff that they need for their insurance application approval process for the calendar year 2021.
The objective is to optimize the total cost for the application approval process by distributing the right number of applications between the FTEs and the vendors while meeting the monthly demand for each state at the same time.


In [1]:
# Importing Libraries 

import pandas as pd
import numpy as np
import math
from pyomo.environ import *

In [2]:
from __future__ import division
from pyomo.opt import SolverFactory

In [3]:
# Reading the data from Excel workbook
filename = 'Staffing+Data.xlsx'
demandData = pd.read_excel(filename, sheet_name='DemandData')
staffAv = pd.read_excel(filename, sheet_name='StaffAvailability')
cost = pd.read_excel(filename, sheet_name='Cost')
serviceRate = pd.read_excel(filename, sheet_name='ServiceRate')

In [4]:
demandData.head()

,State,Month,Demand
0,A,Jan,5240
1,A,Feb,4878
2,A,Mar,5942
3,A,Apr,2297
4,A,May,1992


In [5]:
staffAv.head()

,State,Month,LB,UB,StaffAvPer
0,A,Jan,0.70,0.90,0.81
1,A,Feb,0.65,0.85,0.76
2,A,Mar,0.70,0.80,0.75
3,A,Apr,0.75,0.85,0.80
4,A,May,0.70,0.85,0.78


In [6]:
cost.head()

,State,Month,AnnualSalary,MonthlySalary,UnitOutSourceCost
0,A,Jan,60000,5000.0,180
1,A,Feb,60000,5000.0,180
2,A,Mar,60000,5000.0,180
3,A,Apr,60000,5000.0,180
4,A,May,60000,5000.0,180


In [7]:
serviceRate['MgAppServedPerMonth'][0]

40

## Data Pre-Processing 

#### Creating the required Python data structures for Indexes and Parameters

In [8]:
# Defining Indexes
BankLoc = demandData['State'].unique()
BankLoc = BankLoc.tolist()

Month = demandData['Month'].unique()
Month = Month.tolist()

In [9]:
# Defining Parameters for Demand
demand = demandData.set_index(['State','Month'])['Demand'].to_dict()

In [10]:
# Defining Parameters for Cost
FTE_monthlySalary = cost.set_index(['State','Month'])['MonthlySalary'].to_dict()
unitOutSourceCost = cost.set_index(['State','Month'])['UnitOutSourceCost'].to_dict()

In [11]:
# Defining Parameters for Staff Availability
staffAvPer = staffAv.set_index(['State','Month'])['StaffAvPer'].to_dict()
staffAvPer_LB = staffAv.set_index(['State','Month'])['LB'].to_dict()
staffAvPer_UB = staffAv.set_index(['State','Month'])['UB'].to_dict()

In [12]:
# Defining Parameters for FTE service Rate
FTE_serviceRate = serviceRate['MgAppServedPerMonth'][0]

# Question 1

The company wants to know the optimised staffing recommendations for the business case described. 
Write the mathematical model for the deterministic optimisation problem. Define and explain your decision variables, objective function and the constraint. (Hint: Use months of the year as the model timeline).



##  Mathematical formulation / Pyomo components

---

**Sets:** 

The indexes for the given problem are, <br>
- State, $s \in state$
- Month, $m \in month$

---

**Parameters:** 

- $\text{demand}_{s,m} \text{ - Number of applications expected for 2021}$ <br>
- $\text{FTE_monthlySalary}_{s,m} \text{ - Average Monthly Salary Per Staff}$ <br>
- $\text{unitOutSourceCost}_{s,m} \text{ - Cost per application for Vendor}$ <br>
- $\text{staffAvPer}_{s, m} \text{ - Average availability of an FTE}$ <br>
- $\text{FTE_serviceRate}_{s, m} \text{ - Number of insurance applications that can be processed by an FTE per month with 100% availability}$ <br>


---

**Decision variables:**
- Total number of FTE utilized per State per month <br>

> ${MonthlyFTE}_{s,m} \,\,\,\,\,\,\, \text{where} \ s \in state, m \in month $<br>

- Total number of applications outsourced per State per month <br>

> ${MonthlyOutsource}_{s,m} \,\,\,\,\,\,\, \text{where} \ s \in state, m \in month $<br>


---

**Objective:** <br>

To minimize total cost of processing all the applictions as projected by the Demand.



\begin{align}
\textrm{min} \Bigg[\sum \limits _{s,m} \text{MonthlyFTE}_{s,m}*\text{FTE_monthlySalary}_{s,m}  + \sum \limits _{s,m} \text{MonthlyOutsource}_{s,m}* \text{unitOutSourceCost}_{s,m} \Bigg]
\end{align}
where $s \in state$ and $m \in month$

---

**Constraints:**



*   All the applications need to be processed every Month for each State


>- $\text{MonthlyFTE}_{s,m}*\text{staffAvPer}_{s,m}*\text{FTE_serviceRate}_{s,m}+\text{MonthlyOutsource}_{s,m} =\text{demand}_{s,m}\ \ \ \ \forall {s \in state, m \in month}$


*   Maximum applications outsourced for State A


>- $\text{MonthlyOutsource}_{A,m} \leq\text{demand}_{A,m}*0.3\ \ \ \ \forall {m \in month}$


*   Maximum applications outsourced for State B


>- $\text{MonthlyOutsource}_{B,m} \leq\text{demand}_{B,m}*0.4\ \ \ \ \forall {m \in month}$

----


# Question 2

Code the problem is Python and use any optimization package to solve it. Add comments to your code to explain each step. 

#### Expected output:

Create a data frame containing the number of outsourced applications  and the number of FTEs for each state-month combination. You can choose to have extra columns like staff availability, demand etc. in your dataframe apart from the ones mentioned earlier. 

In [13]:
# Creating a model instance
model = ConcreteModel()

In [14]:
# Define Pyomo sets/Indexes and Parameters -> Already defined in Data Pre-processing

In [15]:
# Decision variables
# FTE Work Force
model.MonthlyFTE = Var(BankLoc, Month, domain=NonNegativeReals)

# No. of applications outsourced
model.MonthlyOutsource = Var(BankLoc, Month, domain=NonNegativeIntegers)

In [16]:
# Defining Objective Function

def obj_rule(model):
    return(sum(model.MonthlyFTE[i,j]*FTE_monthlySalary[i,j] for i in BankLoc for j in Month) + 
           sum(model.MonthlyOutsource[i,j]*unitOutSourceCost[i,j] for i in BankLoc for j in Month))

model.value = Objective(rule=obj_rule, sense=minimize)

In [17]:
# Defining Constraints

# Demand Constraint
def fulfill_demand(model,i,j):
    return (model.MonthlyFTE[i,j]*staffAvPer[i,j]*FTE_serviceRate + model.MonthlyOutsource[i,j]) == demand[i,j]

model.match_demand = Constraint(BankLoc,Month,rule=fulfill_demand)

In [18]:
# Regulatory Constraint for outsourced applications for State A

def reg_restrict_state_A(model,j):
    return (model.MonthlyOutsource['A',j] <= demand['A',j]*0.3)
    
model.reg_state_A = Constraint(Month,rule=reg_restrict_state_A)


# Regulatory Constraint for outsourced applications for State B

def reg_restrict_state_B(model,j):
    return (model.MonthlyOutsource['B',j] <= demand['B',j]*0.4)
    
model.reg_state_B = Constraint(Month,rule=reg_restrict_state_B)

In [19]:
# Invoking the solver
result = SolverFactory('glpk').solve(model)

In [20]:
# Print the value of the objective function
model.value()

17962336.448769882

**`Checkpoint 1:`** Seems like the company has to spend around 17.9 m$ in total for the application approval process.

In [21]:
# Creating dataframe for the results
Staff_Plan = []
for i in BankLoc:
    for j in Month:
        dmd = demand[i,j]
        stAv = staffAvPer[i,j]
        fte = model.MonthlyFTE[i,j].value
        outapp = model.MonthlyOutsource[i,j].value
        Staff_Plan.append([i,j,dmd,stAv,fte,outapp]) # making a list of lists
        
Staff_Plan = pd.DataFrame(Staff_Plan, columns=['State','Month','Demand','Staff_Availability','FTEs','Outsourced_Applications'])
Staff_Plan.head(10)

,State,Month,Demand,Staff_Availability,FTEs,Outsourced_Applications
0,A,Jan,5240,0.81,161.728395,0.0
1,A,Feb,4878,0.76,160.460526,0.0
2,A,Mar,5942,0.75,198.066667,0.0
3,A,Apr,2297,0.80,71.781250,0.0
4,A,May,1992,0.78,63.846154,0.0
5,A,Jun,2275,0.73,77.910959,0.0
6,A,Jul,5334,0.68,137.279412,1600.0
7,A,Aug,3371,0.76,110.888158,0.0
8,A,Sep,3759,0.81,116.018519,0.0
9,A,Oct,3529,0.73,120.856164,0.0


In [22]:
# Writing the results in to an Excel sheet
Staff_Plan.to_excel('Staff_Planning.xlsx', sheet_name='Staff_Plan')

In [23]:
# Percentage of outsourced applications
PercentOutsourcedApp = ((Staff_Plan['Outsourced_Applications'].sum()/Staff_Plan['Demand'].sum())* 100).round(2)
PercentOutsourcedApp

18.31

In [24]:
# Average cost per application
AvgCostPerApp = (model.value()/Staff_Plan['Demand'].sum()).round(2)
AvgCostPerApp

158.55

# Question 3

#### Worst-case and best-case analysis based on the staffs' availability.

Assuming that the distribution is the same across all the states,

#### 3.1 Worst case analysis 

- 3.1.1 What is the optimal number of staff members for the worst case? 

- 3.1.2 What is the percentage of outsourcing for the worst case? 

- 3.1.3 What is the average cost per application for the worst case?


#### 3.2 Best case analysis 

- 3.2.1 What is the optimal number of staff members for the best case? 

- 3.2.2 What is the percentage of outsourcing for the best case? 

- 3.2.3 What is the average cost per application for the best case?


#### Expected output:

For each of the subtasks (3.1 and 3.2) create a data frame containing the number of outsourced applications and the number of FTEs for each state-month combination. You can choose to have extra columns like staff availability, demand etc. in your dataframe apart from the ones mentioned earlier. Also, print the overall average percentage of outsourced applications and the overall average cost per application. 

### 3.1 Worst Case Analysis 

#### 3.1.1 Optimal number of staff members

In [25]:
# Creating a model instance
model = ConcreteModel()

In [26]:
# Define Pyomo sets and Parameters -> Already defined in Data Pre-processing

In [27]:
# Decision variables
# FTE Work Force
model.MonthlyFTE = Var(BankLoc, Month, domain=NonNegativeReals)

# No. of applications outsourced
model.MonthlyOutsource = Var(BankLoc, Month, domain=NonNegativeIntegers)

In [28]:
# Defining Objective Function

def obj_rule(model):
    return(sum(model.MonthlyFTE[i,j]*FTE_monthlySalary[i,j] for i in BankLoc for j in Month) + sum(model.MonthlyOutsource[i,j]*unitOutSourceCost[i,j] for i in BankLoc for j in Month))

model.value = Objective(rule=obj_rule, sense=minimize)

In [29]:
# Defining Constraints

# Demand Constraint
def fulfill_demand(model,i,j):
    return (model.MonthlyFTE[i,j]*staffAvPer_LB[i,j]*FTE_serviceRate + model.MonthlyOutsource[i,j]) == demand[i,j]

model.match_demand = Constraint(BankLoc,Month,rule=fulfill_demand)

In [30]:
# Regulatory Constraint for outsourced applications for State A

def reg_restrict_state_A(model,j):
    return (model.MonthlyOutsource['A',j] <= demand['A',j]*0.3)
    
model.reg_state_A = Constraint(Month,rule=reg_restrict_state_A)


# Regulatory Constraint for outsourced applications for State B

def reg_restrict_state_B(model,j):
    return (model.MonthlyOutsource['B',j] <= demand['B',j]*0.4)
    
model.reg_state_B = Constraint(Month,rule=reg_restrict_state_B)

In [31]:
# Invoking the solver
result = SolverFactory('glpk').solve(model)

In [32]:
# Print the value of the objective function
model.value()

19599482.516788766

**`Checkpoint 2:`** The company has to spend around 19.6 m$ in total for the application approval process if the staffs are working with the minimum availability.

In [33]:
# Creating dataframe for the results
Staff_Plan = []
for i in BankLoc:
    for j in Month:
        dmd = demand[i,j]
        stAv = staffAvPer_LB[i,j]
        fte = model.MonthlyFTE[i,j].value
        outapp = model.MonthlyOutsource[i,j].value
        Staff_Plan.append([i,j,dmd,stAv,fte,outapp]) # making a list of lists
        
Staff_Plan = pd.DataFrame(Staff_Plan, columns=['State','Month','Demand','Staff_Availability_LB','FTEs','Outsourced_Applications'])
Staff_Plan.head(10)

,State,Month,Demand,Staff_Availability_LB,FTEs,Outsourced_Applications
0,A,Jan,5240,0.70,187.142857,0.0
1,A,Feb,4878,0.65,131.346154,1463.0
2,A,Mar,5942,0.70,212.214286,0.0
3,A,Apr,2297,0.75,76.566667,0.0
4,A,May,1992,0.70,71.142857,0.0
5,A,Jun,2275,0.65,61.269231,682.0
6,A,Jul,5334,0.60,155.583333,1600.0
7,A,Aug,3371,0.65,90.769231,1011.0
8,A,Sep,3759,0.70,134.250000,0.0
9,A,Oct,3529,0.65,95.038462,1058.0


#### 3.1.2 Percentage of outsourced applications 

In [34]:
# write your code here
PercentOutsourcedApp = ((Staff_Plan['Outsourced_Applications'].sum()/Staff_Plan['Demand'].sum())* 100).round(2)
PercentOutsourcedApp

35.14

#### 3.1.3 Average cost per application

In [35]:
# write your code here
AvgCostPerApp = (model.value()/Staff_Plan['Demand'].sum()).round(2)
AvgCostPerApp

173.0

### 3.2  Best Case Analysis 

#### 3.2.1 Optimal number of staff members


In [36]:
# Creating a model instance
model = ConcreteModel()

In [37]:
# Define Pyomo sets and Parameters -> Already defined in Data Pre-processing

In [38]:
# Decision variables
# FTE Work Force
model.MonthlyFTE = Var(BankLoc, Month, domain=NonNegativeReals)

# No. of applications outsourced
model.MonthlyOutsource = Var(BankLoc, Month, domain=NonNegativeIntegers)

In [39]:
# Defining Objective Function

def obj_rule(model):
    return(sum(model.MonthlyFTE[i,j]*FTE_monthlySalary[i,j] for i in BankLoc for j in Month) + 
           sum(model.MonthlyOutsource[i,j]*unitOutSourceCost[i,j] for i in BankLoc for j in Month))

model.value = Objective(rule=obj_rule, sense=minimize)

In [40]:
# Defining Constraints

# Demand Constraint
def fulfill_demand(model,i,j):
    return (model.MonthlyFTE[i,j]*staffAvPer_UB[i,j]*FTE_serviceRate + model.MonthlyOutsource[i,j]) == demand[i,j]

model.match_demand = Constraint(BankLoc,Month,rule=fulfill_demand)

In [41]:
# Regulatory Constraint for outsourced applications for State A

def reg_restrict_state_A(model,j):
    return (model.MonthlyOutsource['A',j] <= demand['A',j]*0.3)
    
model.reg_state_A = Constraint(Month,rule=reg_restrict_state_A)


# Regulatory Constraint for outsourced applications for State B

def reg_restrict_state_B(model,j):
    return (model.MonthlyOutsource['B',j] <= demand['B',j]*0.4)
    
model.reg_state_B = Constraint(Month,rule=reg_restrict_state_B)

In [42]:
# Invoking the solver
result = SolverFactory('glpk').solve(model)

In [43]:
# Print the value of the objective function
model.value()

16527535.63793573

**`Checkpoint 3:`** The company has to spend around 16.5 m$ in total for the application approval process if the staffs are working with the maximum availability.

In [44]:
# Creating dataframe for the results
Staff_Plan = []
for i in BankLoc:
    for j in Month:
        dmd = demand[i,j]
        stAv = staffAvPer_UB[i,j]
        fte = model.MonthlyFTE[i,j].value
        outapp = model.MonthlyOutsource[i,j].value
        Staff_Plan.append([i,j,dmd,stAv,fte,outapp]) # making a list of lists
        
Staff_Plan = pd.DataFrame(Staff_Plan, columns=['State','Month','Demand','Staff_Availability_UB','FTEs','Outsourced_Applications'])
Staff_Plan.head(10)

,State,Month,Demand,Staff_Availability_UB,FTEs,Outsourced_Applications
0,A,Jan,5240,0.90,145.555556,0.0
1,A,Feb,4878,0.85,143.470588,0.0
2,A,Mar,5942,0.80,185.687500,0.0
3,A,Apr,2297,0.85,67.558824,0.0
4,A,May,1992,0.85,58.588235,0.0
5,A,Jun,2275,0.80,71.093750,0.0
6,A,Jul,5334,0.75,177.800000,0.0
7,A,Aug,3371,0.85,99.147059,0.0
8,A,Sep,3759,0.90,104.416667,0.0
9,A,Oct,3529,0.80,110.281250,0.0


#### 3.2.2 Percentage of outsourced applications

In [45]:
# write your code here
PercentOutsourcedApp = ((Staff_Plan['Outsourced_Applications'].sum()/Staff_Plan['Demand'].sum())* 100).round(2)
PercentOutsourcedApp

4.11

#### 3.2.3 Average cost per application

In [46]:
# write your code here
AvgCostPerApp = (model.value()/Staff_Plan['Demand'].sum()).round(2)
AvgCostPerApp

145.88

# Question 4

#### Creating Visualisations

Create the following visualisations using your preferred method (i.e. Python, PowerPoint, Power BI, etc.) and add it to your report. 

Use the solution of Q2 to create a stacked column chart that shows the percentage of applications processed by the staff and by the vendor for each month (%staff processed applications+ %vendor processed applications should add up to 100%). 
Create a graph to show how the cost per application increases with respect to any change in the parameters in your analysis.
Hint: Use the cost per application that you calculate in Questions 2 and 3 (i.e., the best case, and the worst case). 

**Note:** You can create the charts in Python or some other visualisation tools and make it a part of your final report directly.

Kindly refer to the final report for visualizations & insights.

#### Thank You!